In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=ToTensor())
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=ToTensor())

batch_size = 64
training_data_loader = DataLoader(training_data, batch_size)
test_data_loader = DataLoader(test_data, batch_size)

# for X, y in test_data_loader:
#     print(f"Shape of X [N, C, H, W]: {X.shape}")
#     print(f"Shape of y: {y.shape} {y.dtype}")
#     break

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()   # A flatten layer that converts the dataset (a 2D image 28x28 pixels) into a 1D vector (784 elements).
        self.linear_relu_stack = nn.Sequential(    # Creates a sequential container that holds a series of layers
            nn.Linear(28*28, 512),    # a FC layer that converts a 784-element vector into a 512-element vector.
            nn.ReLU(),                # activation function (ReLU) to introduce nonlinearity
            nn.Linear(512, 512),      # a FC layer
            nn.ReLU(),                # activation function (ReLU) to introduce nonlinearity
            nn.Linear(512, 10),       # final FC layer that converts a 512-element vector into a 10-element vector.
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


def train(dataloader, model, loss_fn, optimizer, log=True):
    size = len(dataloader.dataset)
    model.train()    # set the model to training mode
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)          # forward pass
        loss = loss_fn(pred, y)  # compute loss
        # Backpropagation
        optimizer.zero_grad()   # Clears old gradients.
        loss.backward()         # backward pass
        optimizer.step()        # update weights

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            if log:
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()               # set the model to evaluation mode
    test_loss, correct = 0, 0
    with torch.no_grad():      # disables gradient calculation, saving memory and speeding up computations.
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

train(training_data_loader, model, loss_fn, optimizer)

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

test(test_data_loader, model, loss_fn)







Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
loss: 2.287117  [   64/60000]
loss: 2.283170  [ 6464/60000]
loss: 2.265362  [12864/60000]
loss: 2.264601  [19264/60000]
loss: 2.246406  [25664/60000]
loss: 2.218616  [32064/60000]
loss: 2.223468  [38464/60000]
loss: 2.188797  [44864/60000]
loss: 2.168612  [51264/60000]
loss: 2.152873  [57664/60000]
Saved PyTorch Model State to model.pth
Test Error: 
 Accuracy: 54.4%, Avg loss: 2.144512 



In [3]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

for i in range(30):
    print(f"Epoch {i+1}\n-------------------------------")
    train(training_data_loader, model, loss_fn, optimizer, log=True)
    test(test_data_loader, model, loss_fn)

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")


D:\Users\Yan\AppData\Local\Temp\ipykernel_7828\201785875.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))


Epoch 1
-------------------------------
loss: 2.148041  [   64/60000]
loss: 2.158906  [ 6464/60000]
loss: 2.131664  [12864/60000]
loss: 2.168809  [19264/60000]
loss: 2.148302  [25664/60000]
loss: 2.126748  [32064/60000]
loss: 2.166982  [38464/60000]
loss: 2.138972  [44864/60000]
loss: 2.142270  [51264/60000]
loss: 2.144313  [57664/60000]
Test Error: 
 Accuracy: 54.4%, Avg loss: 2.144512 

Epoch 2
-------------------------------
loss: 2.148041  [   64/60000]
loss: 2.158906  [ 6464/60000]
loss: 2.131664  [12864/60000]
loss: 2.168809  [19264/60000]
loss: 2.148302  [25664/60000]
loss: 2.126748  [32064/60000]
loss: 2.166982  [38464/60000]
loss: 2.138972  [44864/60000]
loss: 2.142270  [51264/60000]
loss: 2.144313  [57664/60000]
Test Error: 
 Accuracy: 54.4%, Avg loss: 2.144512 

Epoch 3
-------------------------------
loss: 2.148041  [   64/60000]


KeyboardInterrupt: 